In [ ]:
!gcloud auth login    

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=CBAAVf8JibXjVYGsGHQsx8RMvhditU&access_type=offline&code_challenge=PgJXfQCXhuKAIDjJmx6q7YqN2JJlYEeK1soHzWfzQ0Q&code_challenge_method=S256


You are now logged in as [bastinflorian1@gmail.com].
Your current project is [dauphine-437611].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
%pip install -q google-cloud-storage unstructured langchain python-magic sqlalchemy langchain_google_cloud_sql_pg
%pip install -q "unstructured[pptx]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Ingesting in Cloud SQL

In [54]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


In [55]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [ ]:
TABLE_NAME =  "MI_RAG" 

In [58]:
from langchain_google_cloud_sql_pg import PostgresEngine

# Connect to the PostgreSQL database
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD,
)


In [ ]:

try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME,
        vector_size=768,
        )
except ProgrammingError:
    print("Table already created")
    logger.info("✅ Table MI_RAG verified/created successfully.")
except SQLAlchemyError as e:
    logger.error(f"❌ Error creating the table: {e}") 

Table already created


###  Create an embedding to CONVERT documents

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

In [ ]:
from langchain_google_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(  
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)